In [2]:
### Get all key dfs from Cleaned Data
import pandas as pd

df_t = pd.read_csv('Data/Cleaned/TSS metrics.csv')
df_s = pd.read_csv('Data/Cleaned/Sleep_and_recovery.csv')
df_f = pd.read_csv('Data/Cleaned/MFP per day scrapped.csv')
df_g = pd.read_csv('Data/Cleaned/Glucose_daily.csv')
df_gar = pd.read_csv('Data/Cleaned/Garmin_daily.csv')
df_j = pd.read_csv('Data/Cleaned/Journal.csv')
df_w = pd.read_csv('Data/Cleaned/Weight.csv')

# Print the min and the max date of each df
print('TSS metrics: ', df_t['date'].min(),' to ',df_t['date'].max())
print('Sleep and recovery: ',df_s['date'].min(),' to ',df_s['date'].max())
print('MFP per day scrapped: ',df_f['date'].min(),' to ',df_f['date'].max())
print('Glucose daily: ',df_g['date'].min(),' to ',df_g['date'].max())
print('Garmin daily: ',df_gar['date'].min(),' to ',df_gar['date'].max())
print('Journal: ',df_j['date'].min(),' to ',df_j['date'].max())
print('Weight: ',df_w['date'].min(),' to ',df_w['date'].max())


TSS metrics:  2023-05-17  to  2024-05-12
Sleep and recovery:  2024-03-16  to  2024-05-15
MFP per day scrapped:  2024-03-16  to  2024-05-15
Glucose daily:  2024-03-23  to  2024-05-15
Garmin daily:  2024-03-16  to  2024-05-15
Journal:  2024-03-14  to  2024-05-14
Weight:  2024-03-11  to  2024-05-15


In [12]:
# Filter all dfs to the same date range, the one between the min and max date of all dfs except glucose and weight
min_date = max(df_t['date'].min(),df_s['date'].min(),df_f['date'].min(), df_gar['date'].min(), df_j['date'].min())
max_date = min(df_t['date'].max(),df_s['date'].max(),df_f['date'].max(), df_gar['date'].max(), df_j['date'].max())

df_t = df_t[(df_t['date'] >= min_date) & (df_t['date'] <= max_date)]
df_s = df_s[(df_s['date'] >= min_date) & (df_s['date'] <= max_date)]
df_f = df_f[(df_f['date'] >= min_date) & (df_f['date'] <= max_date)]
df_g = df_g[(df_g['date'] >= min_date) & (df_g['date'] <= max_date)]
df_gar = df_gar[(df_gar['date'] >= min_date) & (df_gar['date'] <= max_date)]
df_j = df_j[(df_j['date'] >= min_date) & (df_j['date'] <= max_date)]
df_w = df_w[(df_w['date'] >= min_date) & (df_w['date'] <= max_date)]

# Perform an outter join on all dfs
df = df_t.merge(df_s, on='date', how='outer')
df = df.merge(df_f, on='date', how='outer')
df = df.merge(df_g, on='date', how='outer')
df = df.merge(df_gar, on='date', how='outer')
df = df.merge(df_j, on='date', how='outer')
df = df.merge(df_w, on='date', how='outer')

# Check first and last date of the new df
print('New df: ',df['date'].min(),' to ',df['date'].max())

df.columns

New df:  2024-03-16  to  2024-05-04


Index(['date', 'TSS', 'CTL', 'ATL', 'TSB', 'sleep_time',
       'sleep_score_performance', 'sleep_score_consistency',
       'sleep_score_efficiency', 'sleep_duration', 'sleep_rem', 'sleep_deep',
       'sleep_light', 'sleep_awake', 'recovery_score', 'resting_hr', 'hrv',
       'spo2', 'skin_temp', 'calories_burned', 'carbs', 'fat', 'protein',
       'sodium', 'sugar', 'calories_consumed', 'calories_goal', 'calories_net',
       'calories_consumed_breakfast', 'calories_consumed_lunch',
       'calories_consumed_dinner', 'calories_consumed_snacks', 'mean_glucose',
       'std_glucose', 'max_glucose', 'wake_up_glucose', 'averageStressLevel',
       'restStressPercentage', 'lowStressPercentage', 'mediumStressPercentage',
       'highStressPercentage', 'stressQualifier', 'bodyBatteryHighestValue',
       'bodyBatteryLowestValue', 'bodyBatteryDuringSleep',
       'avoid_processed_foods', 'bed_full', 'sick_or_ill', 'injury', 'alcohol',
       'read_bed', 'stretch', 'screen_bed', 'BMI', 'fat_

In [13]:
# Save to CSV
df.to_csv('Data/Cleaned/Integrated_data.csv', index=False)

print('Integrated data file created')

Integrated data file created


In [3]:
### Upload to Google Sheets

import pandas as pd
from googleapiclient.discovery import build
from google.oauth2 import service_account

# Load service account credentials
creds = service_account.Credentials.from_service_account_file('gsheets key.json')
scoped_credentials = creds.with_scopes(['https://www.googleapis.com/auth/spreadsheets'])

# Authenticate with Google Sheets API
service = build('sheets', 'v4', credentials=scoped_credentials)

# Spreadsheet ID and range for Health Dashboards
SPREADSHEET_ID_input = '197VfZCekvBev0m1vsi8kUHpuO0IoTRA90_bQRGBYYSM'

def Export_Data_To_Sheets(df, sheet_name):
    # Specify the range including the sheet name
    range_name = f"{sheet_name}!A1:ZA1000"
    
    # Fill NaN values with empty strings
    df_filled = df.fillna('')
    
    # Get DataFrame headers and values
    headers = [df_filled.columns.tolist()]
    values = df_filled.values.tolist()
    
    # Concatenate headers with values
    data = headers + values
    
    # Update the spreadsheet with DataFrame values including headers
    response = service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID_input,
        valueInputOption='RAW',
        range=range_name,
        body=dict(
            majorDimension='ROWS',
            values=data)
    ).execute()

# Specify the sheet name
sheet_name = 'Integrated_data'

# Export DataFrame to Google Sheets with specified sheet name
Export_Data_To_Sheets(df, sheet_name)
print('Uploaded to Google Sheets')


NameError: name 'df' is not defined